<a href="https://colab.research.google.com/github/HuyenTrang457/MCM/blob/main/Pre_processing_for_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np


Clean Data:

1. Dùng biểu đồ box_plot để phát hiện các khoảng chứ outlier và lọc outlier.
2. Transform các biến về dạng phù hợp và chuẩn hóa
3. Chuẩn hóa các biến liên tục
4. Chuẩn hóa các biến liên tục
5. Xử lý các giá trị chuỗi trong p1_score và p2_score



In [ ]:
file_path = "/content/Wimbledon_featured_matches.csv"
df = pd.read_csv(file_path)

# Vẽ biểu đồ boxplot để phát hiện và lọc outlier
# Danh sách các cột cần kiểm tra outlier
columns_to_analyze = ["p1_distance_run", "p2_distance_run", "rally_count", "speed_mph"]
# Tạo thư mục để lưu các hình ảnh boxplot
output_dir = "/content/boxplot_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Vẽ boxplot và lưu hình ảnh
for var in columns_to_analyze:
    sns.boxplot(x=df[var], color="darkorange")  # Màu cam cho boxplot
    plt.title(f"Boxplot of {var}", fontsize=14)  # Tiêu đề boxplot
    plt.xlabel(var, fontsize=12)
    plt.grid(axis="x", linestyle="--", alpha=0.6)  # Thêm lưới để dễ quan sát
    plt.savefig(f"{output_dir}/{var}_boxplot.png")  # Lưu hình ảnh vào thư mục
    plt.close()

print(f"Biểu đồ boxplot đã được lưu vào thư mục: {output_dir}")

# 6. Lọc các giá trị ngoại lai (outlier)
# Giữ lại các hàng với điều kiện lọc outlier cho từng cột
df = df[
    (df['p1_distance_run'] <= 100) &
    (df['p2_distance_run'] <= 100) &
    (df['rally_count'] <= 20) &
    (df['speed_mph'] >= 80)
]

# Tạo thư mục để lưu hình ảnh boxplot sau khi lọc
output_dir1 = "/content/boxplot_images1"
if not os.path.exists(output_dir1):
    os.makedirs(output_dir1)

# Vẽ lại boxplot và lưu hình ảnh sau khi lọc
for var in columns_to_analyze:
    sns.boxplot(x=df[var], color="darkorange")  # Màu cam cho boxplot
    plt.title(f"Boxplot of {var} (After Filtering)", fontsize=14)  # Tiêu đề boxplot
    plt.xlabel(var, fontsize=12)  # Nhãn trục x
    plt.grid(axis="x", linestyle="--", alpha=0.6)  # Thêm lưới để dễ quan sát
    plt.savefig(f"{output_dir1}/{var}_boxplot.png")  # Lưu hình ảnh vào thư mục
    plt.close()
# update lại data vào file mới
csv_file_path = '/content/Wimbledon_featured_matches1.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')


FileNotFoundError: [Errno 2] No such file or directory: '/content/Wimbledon_featured_matches.csv'

In [ ]:
# 1. Đọc tệp dữ liệu
file_path = '/content/Wimbledon_featured_matches1.csv'  # Thay bằng đường dẫn chính xác
raw_data = pd.read_csv(file_path)

# 2. Xử lý các giá trị chuỗi trong p1_score và p2_score
def convert_score_to_numeric(score):
    try:
        return float(score)
    except ValueError:
        return 50  # Chuyển giá trị "AD" hoặc bất kỳ chuỗi nào khác thành 50

raw_data['p1_score'] = raw_data['p1_score'].apply(convert_score_to_numeric)
raw_data['p2_score'] = raw_data['p2_score'].apply(convert_score_to_numeric)

# 3. Tạo các biến mới theo bảng đã cung cấp
def process_data(data):
    # 3.1 Ace (giữ nguyên)
    data['Ace'] = data['p1_ace']

    # 3.2 Distance_run
    data['Distance_run'] = data['p1_distance_run'] - data['p2_distance_run']

    # 3.3 Error_rate_of_serve_no.1 (rolling window = 3)
    data['Error_rate_of_serve_no.1'] = data['p1_double_fault'].rolling(window=3, min_periods=1).mean()

    # 3.4 Lead_score
    data['Lead_score'] = data['p1_score'] - data['p2_score']

    # 3.5 Net_pt_won
    data['Net_pt_won'] = data['p1_net_pt_won']

    # 3.6 Rally_count
    data['Rally_count'] = data['rally_count'] * data['server'].apply(lambda x: -1 if x == 'p2' else 1)

    # 3.7 Server_pt
    data['Server_pt'] = data['server'].apply(lambda x: 1 if x == 'p1' else 0)

    # 3.8 Speed_mph
    data['Speed_mph'] = data['speed_mph'].fillna(data['speed_mph'].mean())

    # 3.9 Return_depth (One-Hot Encoding)
    return_depth_dummies = pd.get_dummies(data['return_depth'], prefix='Return_depth')
    data = pd.concat([data, return_depth_dummies], axis=1)

    # Chuyển TRUE/FALSE của One-Hot Encoding thành 0/1
    data = data.replace({True: 1, False: 0})

    # 3.10 Serve_depth (One-Hot Encoding)
    serve_depth_dummies = pd.get_dummies(data['serve_depth'], prefix='Serve_depth')
    data = pd.concat([data, serve_depth_dummies], axis=1)

    # 3.11 Serve_width (One-Hot Encoding)
    serve_width_dummies = pd.get_dummies(data['serve_width'], prefix='Serve_width')
    data = pd.concat([data, serve_width_dummies], axis=1)
    data = data.replace({True: 1, False: 0})


   # 3.13 Game_victor
    data['Game_victor'] = data['game_victor']

    # 3.14 Break Point
    data['Break_Point'] = data['p1_break_pt']

    return data

# Xử lý dữ liệu
processed_data = process_data(raw_data)

# 4. Chuẩn hóa các biến liên tục
continuous_columns = [
    'Distance_run', 'Error_rate_of_serve_no.1', 'Lead_score', 'Net_pt_won',
    'Rally_count', 'Speed_mph'
]
scaler = StandardScaler()
processed_data[continuous_columns] = scaler.fit_transform(processed_data[continuous_columns])

# 5. Lưu dữ liệu đã xử lý vào DataFrame mới
final_columns = ['match_id',
    'Ace', 'Distance_run', 'Error_rate_of_serve_no.1', 'Lead_score',
    'Net_pt_won', 'Rally_count', 'Server_pt', 'Speed_mph',
     'Game_victor', 'Break_Point'
] + list(processed_data.filter(regex='^Return_depth_').columns) + list(processed_data.filter(regex='^Serve_depth_').columns) + list(processed_data.filter(regex='^Serve_width_').columns)

final_data = processed_data[final_columns]

# Hiển thị dữ liệu cuối cùng
print("--- Dữ liệu cuối cùng ---")
print(final_data.head())


--- Dữ liệu cuối cùng ---
              match_id  Ace  Distance_run  Error_rate_of_serve_no.1  \
0  2023-wimbledon-1301    0     -0.394420                       0.0   
1  2023-wimbledon-1301    0     -0.394616                       0.0   
2  2023-wimbledon-1301    0     -1.213620                       0.0   
3  2023-wimbledon-1301    0     -4.852655                       0.0   
4  2023-wimbledon-1301    1     -0.065010                       0.0   

   Lead_score  Net_pt_won  Rally_count  Server_pt  Speed_mph  Game_victor  \
0   -0.053137   -0.290596    -0.473250          0  -1.373634            0   
1   -0.938331   -0.290596    -0.805702          0   0.430143            0   
2   -0.053137   -0.290596     0.191654          0   0.586993            0   
3   -0.938331   -0.290596     3.183722          0   1.371244            0   
4   -0.053137   -0.290596    -0.805702          0  -0.040407            0   

   Break_Point  Return_depth_D  Return_depth_ND  Serve_depth_CTL  \
0            0  

<ipython-input-12-a758c1d8840b>:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})
<ipython-input-12-a758c1d8840b>:55: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})


Để trích xuất các feature có các giá trị tốt hơn cho mô hình ta để ý một số điều sau
1. Speed_mph có giá trị hơi lệch phải cũng như có mối quan hệ phi tuyến với momentum mà ta đang cần tìm. Lí do đơn giản vì tốc độ di chuyển nhanh không có nghĩa momentum đang cao , có khi còn ngược lại. Vì thế để làm mượt hơn mối quan hệ giữa momentum và spees_mph ,chúng em đã dùng kĩ thuật Log Transformation để giải quyết.
2. Chúng em nhận thấy mối quan hệ:

*   Rally dài với tốc độ thấp → trận đấu kéo dài (chiến thuật phòng thủ).
*   Rally ngắn với tốc độ cao → điểm nhanh (chiến thuật tấn công).
Do vậy, tụi em đã biểu diễn mối quan hệ đó bằng cách
data['Interaction_Rally_Speed'] = data['Rally_count'] * data['Speed_mph']